In [16]:
import os
import pandas as pd

data_path = "/mnt/L-HDD/try_dataset/hit_objects_formatted.csv"
df = pd.read_csv(data_path)

In [17]:
df.head()

,unique_id,ID,Time,Type,X,Y,HitSound,Path,Repeat,Length,beatmap_id,MFCC,RMS,beat_length,meter,slider_velocity,sample_set,volume,effects
0,7093,2307456-0,421.0,slider,55,231,0,L|41:89,1,127.5,2307456,[-25.825926 58.306747 -51.43855 24.82565...,0.1341288089752197,326.086957,4.0,-100.0,2.0,45.0,0.0
1,7327,2307456-0,65475.0,circle,311,155,0,0,0,0.0,2307456,[ 6.7267356e+00 7.5252098e+01 -1.3352648e+01 ...,0.3174754679203033,326.086957,4.0,-100.0,2.0,80.0,1.0
2,7326,2307456-0,65312.0,circle,221,27,2,0,0,0.0,2307456,[ 5.54217 72.21061 -1.0093832 44.30...,0.2529509663581848,326.086957,4.0,-100.0,2.0,80.0,1.0
3,7325,2307456-0,65149.0,circle,456,94,2,0,0,0.0,2307456,[ 8.007427 53.24177 6.835449 50.15143...,0.2158940136432647,326.086957,4.0,-100.0,2.0,80.0,1.0
4,7324,2307456-0,64660.0,slider,229,192,0,B|290:206|290:206|335:161|416:180,1,170.0,2307456,[ -6.1584635 74.52028 -4.4221554 37.89...,0.2712808847427368,326.086957,4.0,-100.0,2.0,80.0,1.0


In [19]:
# It's about nobarline. When nobarline is on, it flips the 4. bit which means adding 8.

def correct_effect_value(x):
    if(x > 8):
        return x != 5746
    return 0 if x == 8 else x

df["effects"] = df["effects"].apply(correct_effect_value)

In [20]:
categorical_cols = ["Type", "HitSound", "sample_set", "effects"]
df = pd.get_dummies(df, columns=categorical_cols)

In [21]:
from sklearn.preprocessing import MinMaxScaler

df["RMS"] = df["RMS"].str.replace("[", "").str.replace("]", "")
numerical_cols = ["X", "Y", "beat_length", "meter", "slider_velocity", "volume", "RMS"]
scaler = MinMaxScaler()

df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

In [22]:
df.head()

,unique_id,ID,Time,X,Y,Path,Repeat,Length,beatmap_id,MFCC,...,HitSound_10,HitSound_12,HitSound_14,sample_set_-1.0,sample_set_1.0,sample_set_2.0,sample_set_3.0,effects_-1.0,effects_0.0,effects_1.0
0,7093,2307456-0,421.0,0.109162,0.601036,L|41:89,1,127.5,2307456,[-25.825926 58.306747 -51.43855 24.82565...,...,False,False,False,False,False,True,False,False,True,False
1,7327,2307456-0,65475.0,0.608187,0.404145,0,0,0.0,2307456,[ 6.7267356e+00 7.5252098e+01 -1.3352648e+01 ...,...,False,False,False,False,False,True,False,False,False,True
2,7326,2307456-0,65312.0,0.432749,0.072539,0,0,0.0,2307456,[ 5.54217 72.21061 -1.0093832 44.30...,...,False,False,False,False,False,True,False,False,False,True
3,7325,2307456-0,65149.0,0.890838,0.246114,0,0,0.0,2307456,[ 8.007427 53.24177 6.835449 50.15143...,...,False,False,False,False,False,True,False,False,False,True
4,7324,2307456-0,64660.0,0.448343,0.500000,B|290:206|290:206|335:161|416:180,1,170.0,2307456,[ -6.1584635 74.52028 -4.4221554 37.89...,...,False,False,False,False,False,True,False,False,False,True


In [23]:
import re
import numpy as np
def parse_mfcc(mfcc_str):
    mfcc_str = mfcc_str.replace("[", "").replace("]", "")
    mfcc_values = re.split(r'\s+', mfcc_str.strip())
    return np.array(mfcc_values, dtype=np.float32)

df["MFCC"] = df["MFCC"].apply(parse_mfcc)

In [24]:
mfcc_length = 20

for i in range(mfcc_length):
    df[f"MFCC_{i+1}"] = df["MFCC"].apply(lambda x: x[i])

In [25]:
df = df.drop(columns=["unique_id", "beatmap_id", "MFCC"])

In [27]:
df["Path"].apply(lambda x: len(str(x))).max()

np.int64(1150)

In [28]:
def reduce_extra(x):
    if(pd.isna(x)):
        return 0
    elif not '|' in x:
        return x
    coordinates_str = x[2:].split('|')
    coords = [list(map(int, point.split(':'))) for point in coordinates_str]

    threshold = 3
    filtered_coords = [x[0],coords[0]]
    for i in range(1, len(coords)):
        prev_point = filtered_coords[-1]
        curr_point = coords[i]
        distance = np.linalg.norm(np.array(curr_point) - np.array(prev_point))
        if distance > threshold:
            filtered_coords.append(curr_point)
    return filtered_coords
    
# new_df = df[df["Extra"].apply(lambda x: len(str(x))) > 5000]
df["Extra_Splitted"] = df["Path"].apply(reduce_extra)
df

,ID,Time,X,Y,Path,Repeat,Length,RMS,beat_length,meter,...,MFCC_12,MFCC_13,MFCC_14,MFCC_15,MFCC_16,MFCC_17,MFCC_18,MFCC_19,MFCC_20,Extra_Splitted
0,2307456-0,421.000000,0.109162,0.601036,L|41:89,1,127.5,0.186756,0.228801,0.625,...,6.726282,4.476868,-1.645416,-8.575047,-8.523077,-11.832512,-8.327502,-7.100827,-5.237866,"[L, [41, 89]]"
1,2307456-0,65475.000000,0.608187,0.404145,0,0,0.0,0.442624,0.228801,0.625,...,14.605858,1.398154,-0.991041,-9.197731,8.093043,-4.489946,9.594962,5.311083,-1.734651,0
2,2307456-0,65312.000000,0.432749,0.072539,0,0,0.0,0.352577,0.228801,0.625,...,11.322751,-4.193770,5.073362,-6.835540,-8.224681,-13.275893,-1.678725,-4.901132,-5.665561,0
3,2307456-0,65149.000000,0.890838,0.246114,0,0,0.0,0.300863,0.228801,0.625,...,-1.262928,-15.618279,-4.638806,-4.004638,0.610316,-11.848320,-1.910634,-2.835737,-3.585583,0
4,2307456-0,64660.000000,0.448343,0.500000,B|290:206|290:206|335:161|416:180,1,170.0,0.378158,0.228801,0.625,...,5.769188,2.647124,2.418926,-0.590245,-4.559304,-13.328683,-3.516401,-2.819119,-2.787056,"[B, [290, 206], [335, 161], [416, 180]]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179158,2333780-4,61879.000000,0.368421,0.779793,P|248:300|300:256,1,120.0,0.418339,0.263016,0.625,...,8.678497,6.565650,11.228224,9.392997,6.691104,-2.179216,0.551466,-4.256234,6.310300,"[P, [248, 300], [300, 256]]"
179159,2333780-4,61504.000000,0.150097,0.893782,0,0,0.0,0.546236,0.263016,0.625,...,18.657139,8.465160,15.762417,-1.111063,-1.997209,-14.861074,-11.058609,11.255178,-3.289972,0
179160,2333780-4,61129.000000,0.196881,0.593264,0,0,0.0,0.407642,0.263016,0.625,...,8.783878,1.743605,-2.679118,-9.824715,-0.185903,-8.726190,-9.879675,-3.101683,0.904114,0
179161,2333780-4,64129.000000,0.883041,0.541451,0,0,0.0,0.262038,0.263016,0.625,...,-10.669119,-2.484383,-7.079469,-13.567419,-7.989351,-17.162685,-11.069699,-12.846676,0.927126,0


In [15]:
df["Extra"]

0                               7290
1          B|380:278|353:364|471:359
2                          L|253:285
3                                NaN
4                                NaN
                     ...            
8639172                          NaN
8639173                          NaN
8639174                          NaN
8639175                          NaN
8639176                          NaN
Name: Extra, Length: 8639177, dtype: object